In [13]:
# this code results in extremely low accuracy of .2
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
# Enable mixed precision for faster training
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


2025-01-27 21:09:56.918747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-27 21:09:56.957023: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-27 21:09:56.960418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-27 21:09:56.964163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# Constants
IMG_SIZE = 224  # EfficientNetV2 prefers a larger image size
BATCH_SIZE = 64
EPOCHS = 50
AUGMENTED_DATASET_DIR = "/home/natalyagrokh/img_datasets/combo_ferck_dataset_1"  # Augmented dataset directory
NUM_CLASSES = 8  # Number of unique emotions after merging datasets

In [4]:
# Clean hidden or unexpected directories
def clean_hidden_directories(directory):
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if class_name.startswith('.') or not os.path.isdir(class_path):
            print(f"Removing unexpected directory: {class_path}")
            os.rmdir(class_path)  # Remove empty hidden folders

clean_hidden_directories(AUGMENTED_DATASET_DIR)

In [5]:
# Step 1: Calculate Class Weights
def calculate_class_distribution(dataset_dir):
    distribution = {}
    for class_name in os.listdir(dataset_dir):
        # Skip hidden folders (like .ipynb_checkpoints)
        if class_name.startswith('.'):
            continue

        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            num_images = len([
                img for img in os.listdir(class_path)
                if img.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
            ])
            distribution[class_name] = num_images
    return distribution

# Get the class distribution from the dataset directory
new_distribution = calculate_class_distribution(AUGMENTED_DATASET_DIR)
print("New Class Distribution:", new_distribution)

# Generate the labels and counts
labels = list(new_distribution.keys())  # Class names
counts = np.array(list(new_distribution.values()))  # Number of images per class

# Create a mapping of class names to indices
class_to_index = {label: i for i, label in enumerate(labels)}
print("Class to Index Mapping:", class_to_index)

# Create the `y` array (class labels repeated by the number of images in each class)
y = []
for class_name, count in new_distribution.items():
    y.extend([class_to_index[class_name]] * count)
y = np.array(y)

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y),  # Unique class indices
    y=y  # Repeated class labels
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

New Class Distribution: {'disgust': 2896, 'contempt': 216, 'fear': 5196, 'sadness': 6161, 'surprise': 4251, 'anger': 5088, 'neutral': 6198, 'happiness': 9196}
Class to Index Mapping: {'disgust': 0, 'contempt': 1, 'fear': 2, 'sadness': 3, 'surprise': 4, 'anger': 5, 'neutral': 6, 'happiness': 7}
Class Weights: {0: 1.692075276243094, 1: 22.68634259259259, 2: 0.9430812163202463, 3: 0.7953660120110372, 4: 1.1527287697012467, 5: 0.9630994496855346, 6: 0.7906179412713779, 7: 0.5328675511091779}


In [6]:
# Step 2: Load Dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    AUGMENTED_DATASET_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE
)

# Split into train and test datasets (80-20 split)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

Found 39202 files belonging to 8 classes.


2025-01-27 21:10:04.109103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-27 21:10:04.109645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-27 21:10:04.114608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-27 21:10:04.117918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [7]:
# Step 3: Preprocess and Augment Training Data
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.grayscale_to_rgb(image)  # Convert grayscale to RGB for EfficientNet
    return image, label  # Keep labels as integers


data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
])

train_dataset = train_dataset.map(preprocess_image).map(
    lambda x, y: (data_augmentation(x), y)
)
test_dataset = test_dataset.map(preprocess_image)

# Prefetch for efficiency
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
# Step 4: Define the Model
base_model = EfficientNetV2B0(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze the base model initially

model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.4),  # Regularization
    Dense(NUM_CLASSES, activation="softmax", dtype="float32")
])

In [9]:
# Step 5: Compile the Model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  # Use sparse categorical loss
    metrics=["accuracy"]
)

In [10]:
# Step 6: Fine-Tune the Model
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Freeze all but the last 50 layers
    layer.trainable = False

In [11]:
# Step 7: Callbacks
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=7,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

In [12]:
# Step 8: Train the Model with Class Weights
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weights  # Pass the computed weights
)

Epoch 1/50


2025-01-27 21:10:36.391987: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2025-01-27 21:10:37.128694: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x557d97d44c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-27 21:10:37.128750: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-01-27 21:10:37.138821: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-27 21:10:37.343919: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


490/490 [==============================] - 331s 657ms/step - loss: 2.2538 - accuracy: 0.1885 - val_loss: 2.0827 - val_accuracy: 0.2360 - lr: 9.9486e-04
Epoch 2/50
490/490 [==============================] - 313s 639ms/step - loss: 2.2504 - accuracy: 0.1953 - val_loss: 2.1131 - val_accuracy: 0.2344 - lr: 9.8974e-04
Epoch 3/50
490/490 [==============================] - 311s 635ms/step - loss: 2.2475 - accuracy: 0.1967 - val_loss: 2.1597 - val_accuracy: 0.2351 - lr: 9.8464e-04
Epoch 4/50
490/490 [==============================] - ETA: 0s - loss: 2.2436 - accuracy: 0.1927  
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0004897856852039695.
490/490 [==============================] - 313s 638ms/step - loss: 2.2436 - accuracy: 0.1927 - val_loss: 2.1391 - val_accuracy: 0.1325 - lr: 9.7957e-04
Epoch 5/50
490/490 [==============================] - 317s 646ms/step - loss: 2.2451 - accuracy: 0.1916 - val_loss: 2.0907 - val_accuracy: 0.2346 - lr: 9.7453e-04
Epoch 6/50
490/490 [=============

In [ ]:
# Step 9: Save the Model
model.save("efficientnet_emotion_model_augmented_with_weights.keras")